In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import functools
import operator
from urllib.parse import urlparse

import orchest
import requests
import tld
from bs4 import BeautifulSoup

from utils import fetch_urls_in_parallel

In [4]:
def is_valid_url(url):
    try:
        urlparse(url)
        return True
    except Exception:
        return False

In [5]:
def try_get_tld(url):
    try:
        return tld.get_tld(url)
    except Exception:
        return ""

In [6]:
def extract_external_links(html, tld_filter):
    soup = BeautifulSoup(html, features="html.parser")
    a_s = soup.find_all("a")
    return [
        a.attrs["href"]
        for a in a_s
        if "href" in a.attrs
        and "http" in a.attrs["href"]
        and is_valid_url(a.attrs["href"])
        and try_get_tld(a.attrs["href"]) == tld_filter
    ]

In [7]:
def get_random_urls(count=2, lang="nl"):

    random_pages = {
        "nl": "https://nl.wikipedia.org/wiki/Speciaal:Willekeurig",
        "de": "https://de.wikipedia.org/wiki/Spezial:Zuf%C3%A4llige_Seite",
    }

    urls = [random_pages[lang] for i in range(count)]
    resps = fetch_urls_in_parallel(urls, 10)

    links = [
        extract_external_links(resp.content, lang) for resp in resps if resp is not None
    ]
    links = functools.reduce(operator.iconcat, links, [])

    return links

In [8]:
nl_links = get_random_urls(count=orchest.get_step_param("random_page_count"), lang="nl")
de_links = get_random_urls(count=orchest.get_step_param("random_page_count"), lang="de")

e

e

e

e

In [9]:
print("Found %d Dutch domains." % len(nl_links))
print("Found %d German domains." % len(de_links))

Found 48 Dutch domains.
Found 87 German domains.


In [10]:
# print(nl_links)
# print("*" * 20)
# print(de_links)

In [11]:
orchest.output({"nl": nl_links, "de": de_links}, name="domains")